In [11]:
import numpy as np
import pandas as pd
import re
import pickle
from termcolor import colored
import warnings
warnings.filterwarnings("ignore")

import spacy
nlp=spacy.load('es_core_news_sm')

from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

In [2]:
vectorizador_carreras = pickle.load(open("vectorizer/vectorizador_carreras.pkl","rb"))
vectorizador_w5 = pickle.load(open("vectorizer/vectorizador_w5.pkl","rb"))
vectorizador_intents = pickle.load(open("vectorizer/vectorizador_intents.pkl","rb"))
vectorizador_sub_intents = pickle.load(open("vectorizer/vectorizador_sub_intents.pkl","rb"))

In [3]:
modelo_carreras = pickle.load(open("models/modelo_carreras.sav","rb"))
modelo_w5 = pickle.load(open("models/modelo_w5.sav","rb"))
modelo_intents = pickle.load(open("models/modelo_intents.sav","rb"))
modelo_sub_intents = pickle.load(open("models/modelo_sub_intents.sav","rb"))

In [4]:
respuestas = pd.read_csv('data/tbl_respuestas.csv', sep=',', encoding='utf_8')
respuestas.drop("Modalidad", axis=1, inplace=True)
print(respuestas.Intencion.unique())
print(respuestas.SubIntencion.unique())
print(respuestas.Carrera.unique())
print(respuestas.w5.unique())

['generalidades' 'plan' 'tramites' 'pagos' 'trabajo']
['todas' 'preinscripcion' 'cies' 'requisitos' 'equivalencias' 'medios'
 'precio']
['IA' 'MM' 'DG' 'Pub' 'RRPP' 'GCF' 'HyS' 'Log' 'RRHH' 'TUR' 'Agro' 'CI'
 'ADMIN' 'MKT' '3D' 'INF' 'Rob' 'Svirt' 'todas']
['todas' 'como' 'quien' 'que' 'cuando' 'donde']


### PreProcesar y Corregir Documentos

In [5]:
def PreProcesar_carrera(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    fc = open('data/stopwords.txt', 'r', encoding='utf8')
    stopwords = fc.read().split('\n')
    fc.close()
    
    fc1 = open('data/stopwords_carreras.txt', 'r', encoding='utf8')
    stopwords1 = fc1.read().split('\n')
    stopwords1=[x.lower() for x in stopwords1]

    fc1.close()
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    oraciones = [[palabra for palabra in oracion if palabra[2] in stopwords1] for oracion in oraciones]

    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
        #print(oraciones)
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        if oracion == "":
            oracion = "not_match"
        Corpus_Procesado.append(oracion)
    
    return Corpus_Procesado

def Corregir_Documentos_carrera(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar_carrera(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

def carrera_final(entrada):
    
    vari_df_textos_carr = entrada.copy()
    carrera_corr = Corregir_Documentos_carrera(vari_df_textos_carr,['oracion'],False,True,True)

    carrera_corr.loc[carrera_corr.oracion == "not_match"] = 'todas'
    for tipo in carrera_corr.oracion:
        if tipo == 'todas':
            carrera2 = ['todas']
            carrera = 1.
        else:
            array_carreras=vectorizador_carreras.transform([carrera_corr['oracion'][0][:-1]])
            carrera = sorted(list(modelo_carreras.predict_proba(array_carreras)[0]))[-1]
            carrera2 = modelo_carreras.predict(array_carreras)
    if carrera < .5: 
        carrera2 = ['todas']
            
    return carrera2, carrera

#carrera_final(df_textos)

#####################################################################################################

In [6]:
#####################################################################################################
def PreProcesar_w5(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    ww = open('data/stopwords_sin_w5.txt', 'r', encoding='utf8')
    stopwords = ww.read().split('\n')
    #stopwords=[x.lower() for x in stopwords]
    ww.close()
    
    w5_posibles = list(respuestas.w5.unique())
    
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    oraciones = [[palabra for palabra in oracion if palabra[2] in w5_posibles] for oracion in oraciones]
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        if oracion == "":
            oracion = "not_match"
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos_w5(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar_w5(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    #df_textos.loc[df_textos.index == 0, "oracion"] = 'todas'
    return df_textos


def w5_final(entrada):
    
    vari_df_textos_w5 = df_textos.copy()
    w5_corr = Corregir_Documentos_w5(vari_df_textos_w5,['oracion'],False,True,True)

    w5_corr.loc[w5_corr.oracion == "not_match"] = 'todas'
    for tipo in w5_corr.oracion:
        if tipo == 'todas':
            w5_2 = ['todas']
            w5 = 1.
        else:
            array_w5 = vectorizador_w5.transform([w5_corr['oracion'][0][:-1]])
            w5 = sorted(list(modelo_w5.predict_proba(array_w5)[0]))[-1]
            w5_2 = modelo_w5.predict(array_w5)        
    if w5 < .5: 
        w5_2 = ['todas']
        
    return w5_2, w5

#w5_final(df_textos)

In [7]:
#####################################################################################################
def PreProcesar_intents(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    ww = open('data/stopwords_intents.txt', 'r', encoding='utf8')
    stopwords = ww.read().split('\n')
    stopwords=[x.lower() for x in stopwords]
    ww.close()
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        if oracion == "":
            oracion = "not_match"
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos_intents(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar_intents(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

def inte_final(entrada):
    
    vari_df_textos_inte = df_textos.copy()
    inte_corr = Corregir_Documentos_intents(vari_df_textos_inte,['oracion'],False,True,True)
    inte_corr.loc[inte_corr.oracion == "not_match"] = 'generalidades'
    for tipo in inte_corr.oracion:
        if tipo == 'generalidades':
            intents_2 = ['generalidades']
            intents = 1.
        else:
            array_intents = vectorizador_intents.transform([inte_corr['oracion'][0][:-1]])
            intents = sorted(list(modelo_intents.predict_proba(array_intents)[0]))[-1]
            intents_2 = modelo_intents.predict(array_intents)

    if intents < .5: 
        intents_2 = ['generalidades']
            
    return intents_2, intents

#inte_final(df_textos)

def sub_final(entrada):
    
    vari_df_textos_sub = df_textos.copy()
    sub_corr = Corregir_Documentos_intents(vari_df_textos_sub,['oracion'],False,True,True)
    sub_corr.loc[sub_corr.oracion == "not_match"] = 'todas'
    for tipo in sub_corr.oracion:
        if tipo == 'todas':
            sub_intents_2 = ['todas']
            sub_intents = 1.
        else:
            array_sub_intents=vectorizador_sub_intents.transform([sub_corr['oracion'][0][:-1]])
            sub_intents = sorted(list(modelo_sub_intents.predict_proba(array_sub_intents)[0]))[-1]
            sub_intents_2 = modelo_sub_intents.predict(array_sub_intents)
            
    if sub_intents < .5: 
        sub_intents_2 = ['todas']
        
    return sub_intents_2, sub_intents

#sub_final(df_textos)

### Listo

In [ ]:
on = True
info = False
n = 0
cambio = True
while on:
    
    print(colored("Ingrese la pregunta a continuacion:\n","blue"))
    txt=input(colored("Tu: ","green"))

    df_textos = pd.DataFrame(columns = ['oracion'])
    df_textos.loc[0] = [txt]
    
    listita = [inte_final(df_textos)[0][0], sub_final(df_textos)[0][0],
           carrera_final(df_textos)[0][0], w5_final(df_textos)[0][0]]
    df_textos_listo = pd.DataFrame(columns = ['Intencion',"SubIntencion","Carrera","w5"])
    df_textos_listo.loc[0] = listita
    
    respu = respuestas[((respuestas.Intencion==df_textos_listo["Intencion"].values[0])|(respuestas.Intencion=="generalidades"))\
                & ((respuestas.SubIntencion==df_textos_listo["SubIntencion"].values[0])|(respuestas.SubIntencion=="todas"))\
                & ((respuestas.Carrera==df_textos_listo["Carrera"].values[0])|(respuestas.Carrera=="todas"))\
                & ((respuestas.w5==df_textos_listo["w5"].values[0])|(respuestas.w5=="todas"))]
    print("Respuesta:")
    if len(respu.index) < 1: print(colored("No he podido entender la pregunta. ¿Podrias reformularla por favor? ","blue"))                
    else: print(str(respu["Respuesta"].values[n]))
        
    if info:
        print(colored("\ndeteccion:\n","red"),df_textos_listo)
        print(colored("\nclasificacion:\n","red"),respu[["Intencion","SubIntencion","Carrera","w5"]],"\n")
    
    while cambio:
        try:
            rta=input(colored("\nLa respuesta fue acorde a lo solicitado? s/n \n","blue"))
            if rta=="s":
                cambio=False
                n = 0
            elif rta=="n":
                n = n+1
                print(colored("Tal vez esta respuesta sea mas oportuna:\n","blue"))
                print(str(respu["Respuesta"].values[n]))
            elif (rta != "s" )| (rta != "n"):
                print(colored("La opcion ingresada es incorrecta \n","blue"))
        except Exception: 
            print("No he podido encontrar una respuesta acorde. Por favor intente reformular la pregunta.")
            cambio=False
    
    rta=input(colored("Quiere realizar otra pregunta? s/n \n","blue"))
    if rta=="s":
        on=True
        cambio = True
        n = 0
    elif rta=="n":
        on=False
    elif (rta != "s" )| (rta != "n"):
        print(colored("La opcion ingresada es incorrecta \n","blue"))
    if on == False:
        print("\nAdios!")

In [9]:
#que requisitos necesito para inscribirme